In [209]:
kittens = open('./trending_today.in')

In [210]:
lines = kittens.readlines()

In [211]:
lines = [l.strip() for l in lines]
current_line = 0

In [212]:
videos_count, endpoints_count, requests_descriptions_count, caches_count, cache_size = [int(l) for l in lines[current_line].split(' ')]
current_line += 1

In [213]:
video_sizes = [int(l) for l in lines[current_line].split()]
current_line += 1

In [214]:
videos_map = dict()
for i in range(videos_count):
    videos_map[i] = video_sizes[i]

latencies = dict()

In [215]:
for ec in range(endpoints_count):
    endpoint_dc_latency, caches_amount = [int(l) for l in lines[current_line].split()]
    current_line += 1
    latencies[(ec, -1)] = endpoint_dc_latency

    for ca in range(caches_amount):
        cache_index, latency = [int(l) for l in lines[current_line].split()]
        current_line += 1

        latencies[(ec, cache_index)] = latency


In [216]:
requests_list = []
for i in range(requests_descriptions_count):
    video_id, endpoint_id, requests_count = [int(l) for l in lines[current_line].split()]
    current_line += 1

    requests_list.append((video_id, endpoint_id, requests_count))

In [217]:
# requests_list

In [218]:
endpoints_to_sources = dict()

for (endpoint, source_id),v in latencies.items():
    if endpoint in endpoints_to_sources:
        endpoints_to_sources[endpoint].add(source_id)
    else:
        endpoints_to_sources[endpoint] = set([source_id])

In [219]:
# source2videos = {0: set([2]), 1: set([3, 1]), 2: set([0, 1])}

In [220]:
def score(source2videos):
    score = 0
    for video, endpoint, request in requests_list:
        sources = endpoints_to_sources[endpoint]
        lats = []
        for source in sources:
            if source != -1 and video in source2videos[source]:
                lats.append(latencies[(endpoint, source)])
            else: 
                lats.append(latencies[(endpoint, -1)])
        score += request * (latencies[(endpoint, -1)] - min(lats))
    return score

In [221]:
import random
best_source2videos = None
best_score = 0
for j in range(100):
    source2videos = {}
    videos = [i for i in videos_map.keys()]
    for source in range(caches_count):
        current_source_set = set()
        free = cache_size
        current_chosen_size = 0
        while free - current_chosen_size >= 0:
            chosen = random.choice(videos)
            current_chosen_size = videos_map[chosen]
            if free - current_chosen_size >= 0:
                current_source_set.add(chosen)
                free -= current_chosen_size
        source2videos[source] = current_source_set
    current_score = score(source2videos)
    if current_score > best_score:
        best_score = current_score
        best_source2videos = source2videos

# print(best_source2videos)
# score(best_source2videos)

KeyboardInterrupt: 

In [ ]:
with open("./trending_today.out", "w") as out:
    n = len(best_source2videos)
    out.write(str(n))
    out.write('\n')
    for source, videos in best_source2videos.items():
        out.write(str(source) + " " + " ".join([str(i) for i in videos]))
        out.write("\n")